In [ ]:
# -*- coding: utf-8 -*-
""" BEP ANALYSIS
Created on Fri Jan 12 21:24:29 2024

@author: X1 Carbon Gen 7
"""

import matplotlib.pyplot as plt

Units = 1000
SellPrice = 2000
Variable = 800
Fixed = float(input('Fixed cost for this year is: ' ))
TargetProfit = float(input('Target Profit for this year is: ' ))

# -------------------------------------------- in Power Point
# Your units is: 1000
# Your selling price per unit is: 2000
# Variable cost is: 800
# Fixed cost for this year is: 1200000
# Target Profit for this year is: 100000

# -------------------------------------------- in Excel
# Your units is: 2000
# Your selling price per unit is: 60
# Variable cost is: 45
# Fixed cost for this year is: 240000
# Target Profit for this year is: 100000

#PYTHON IS EASY LANGUAGES PROGRAMMING FOR HUMAN 

#C+ C++ JAVA PYSCAL => HARD NOT HUMAN => EASY FRIENDLY COMPUTER

#COMPUTER---------------------------------HUMAN
#C+--------------------------------------PYTHON


CM_Ratio = SellPrice-Variable  # Contribution Margin Ratio 
CM = Units*SellPrice - Units*Variable # Contribution Margin

print (f'\n\n Your Contribution Margin per unit is:  {CM:.2f}' )
NC = CM - Fixed # Net income
print (f'\n\n Your Net Income per unit is:  {NC:.2f}' )
NumberofUnits = (Fixed + TargetProfit) / CM_Ratio

BEP = Fixed/(SellPrice-Variable) # Break-Even Point

BEPS = Fixed/(CM_Ratio/SellPrice) # Break-Even Point in Sales

print (f'\n Your break even point in units is:  {BEP:.2f} ' 
       + f'\n\n Your break even point in total sales dollars is:  {BEPS:.2f}')


# Fixed Costs
plt.plot([0, Fixed],[Fixed, Fixed + 2 * BEP], color='red', label='Fixed Costs')
# Total Costs (Fixed + Variable)
plt.plot([0, 2 * BEP],[Fixed, Fixed + Variable * 2 * BEP], color='orange',label='Total Costs (Fixed + Variable)')
# Revenue
plt.plot([0, 2 * BEP],[0, SellPrice * 2 * BEP], color='green',label='Revenue')

# Break-Even Point
plt.plot([BEP]*2,[0, BEPS], color='black', linestyle='--', lw=1 ,marker = 'o', markerfacecolor = 'red', markersize = 6)
plt.plot([0, BEP],[BEPS]*2, color='black', linestyle='--', lw=1 ,marker = 'o', markerfacecolor = 'red', markersize = 6)

plt.scatter(NumberofUnits, Fixed+BEPS+TargetProfit, c ="blue", linewidths = 2, marker ="*", label='Target Profit: 100.000 (USD)')

plt.legend()
plt.title('Break-Even Analysis by Rinez')
plt.ylabel('Sales in Dollars')
plt.xlabel('Units Sold')

plt.ylim(bottom=0)
plt.xlim(left=0)
plt.xlim(right=2*BEP)
plt.show()


In [2]:
from datetime import datetime, timedelta, date
import pandas as pd
import pyodbc 

# =============================================================================
# Connect to SQL Server STEP_2
# =============================================================================
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)


# =============================================================================
# Transform Data with SQL Server
# =============================================================================
df_RFM = pd.read_sql_query('''WITH RFM_Base 
    AS
    (
        SELECT a.Customer_ID, b.Customer_Name AS CustomerName,
        DATEDIFF(DAY, MAX(a.Order_Date), CONVERT(DATE, GETDATE())) AS Recency_Value,
        COUNT(DISTINCT a.Order_Date) AS Frequency_Value,
        ROUND(SUM(a.Sales), 2) AS Monetary_Value
      FROM [vti_dataset].[dbo].[sales] AS a
      INNER JOIN [vti_dataset].[dbo].[customer] AS b ON a.Customer_ID = b.Customer_ID
      GROUP BY a.Customer_ID, b.Customer_Name
    )
    -- SELECT * FROM RFM_Base
    , RFM_Score 
    AS
    (
      SELECT *,
        NTILE(5) OVER (ORDER BY Recency_Value DESC) as R_Score,
        NTILE(5) OVER (ORDER BY Frequency_Value ASC) as F_Score,
        NTILE(5) OVER (ORDER BY Monetary_Value ASC) as M_Score
      FROM RFM_Base
    )
    -- SELECT * FROM RFM_Score
    , RFM_Final
    AS
    (
    SELECT *,
      CONCAT(R_Score, F_Score, M_Score) as RFM_Overall
      -- , (R_Score + F_Score + M_Score) as RFM_Overall1
      -- , CAST(R_Score AS char(1))+CAST(F_Score AS char(1))+CAST(M_Score AS char(1)) as RFM_Overall2
    FROM RFM_Score
    )
    -- SELECT * FROM RFM_Final
    SELECT f.*, s.Segment
    FROM RFM_Final f
    JOIN [segment scores] s ON f.RFM_Overall = s.Scores
    ; ''', conn)

df_sales = pd.read_sql_query('SELECT * FROM [vti_dataset].[dbo].[sales] ', conn)
df_customers = pd.read_sql_query('SELECT * FROM [vti_dataset].[dbo].[customer] ', conn)
df_segmentScores = pd.read_sql_query('SELECT * FROM [vti_dataset].[dbo].[segment scores]', conn)
# =============================================================================
# Transform Data with Python
# =============================================================================

C:\Users\Admin-s\AppData\Local\Temp\ipykernel_13568\3751359577.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_RFM = pd.read_sql_query('''WITH RFM_Base


In [ ]:
# PYTHON PANDAS STEP_3
df_sales.info()
df_customers.info()
df_segmentScores.info()

df_RFM.info()
df_RFM['CustomerName'].nunique()
# =============================Recency=========================================
# = RFM library => 
 
df_sales['Order_Date'] = pd.to_datetime(df_sales['Order_Date']) ###DATETIME

df_user = df_customers[['Customer_ID','Customer_Name']] ### 2 COT NHO CUA DF_CUSTOMER => TAO 1 CAI BANG MOI
df_user.columns = ['CustomerID','CustomerName'] 

df_maxPurchase = df_sales.groupby('Customer_ID').Order_Date.max().reset_index() #PANDAS GROUP BY
df_maxPurchase.columns = ['CustomerID','MaxPurchaseDate']

df_maxPurchase['Recency Value'] = (datetime.now() - df_maxPurchase['MaxPurchaseDate']).dt.days #DATETIME.NOW - 

df_RFMOverall = pd.merge(df_user, df_maxPurchase[['CustomerID','Recency Value']], on='CustomerID') #JOIN

df_RFMOverall['Recency Value'].describe()

# =============================Frequency=======================================

df_frequency = df_sales.groupby('Customer_ID').Order_Date.nunique().reset_index()
df_frequency.columns = ['CustomerID','Frequency Value']

df_RFMOverall = pd.merge(df_RFMOverall, df_frequency, on='CustomerID')

df_RFMOverall['Frequency Value'].describe()

# =============================Monetery Value==================================

df_monetery = df_sales.groupby('Customer_ID').Sales.sum().reset_index()

df_monetery.columns = ['CustomerID','Monetery Value']

df_RFMOverall = pd.merge(df_RFMOverall, df_monetery, on='CustomerID')

df_RFMOverall['Monetery Value'].describe()

# =============================RFM-Score==================================

df_RFMOverall['R-Score'] = pd.qcut(df_RFMOverall['Recency Value'], 5, labels=[5,4,3,2,1]).astype(str)
df_RFMOverall['F-Score'] = pd.qcut(df_RFMOverall['Frequency Value'], 5, labels=[1,2,3,4,5]).astype(str)
df_RFMOverall['M-Score'] = pd.qcut(df_RFMOverall['Monetery Value'], 5, labels=[1,2,3,4,5]).astype(str)

df_RFMOverall['RFM_Overall'] = df_RFMOverall['R-Score'] + df_RFMOverall['F-Score'] + df_RFMOverall['M-Score']

df_RFMOverall.info()
df_segmentScores.info()

# =============================Segment name==================================

df_segmentScores['Scores'] = df_segmentScores['Scores'].astype(str)

df_RFMOverall = df_RFMOverall.merge(df_segmentScores, how='inner', left_on = 'RFM_Overall', right_on = 'Scores')

df_RFMOverall = df_RFMOverall.iloc[:, :-1]
# df_RFMOverall.drop('Scores', axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Order_Line   9994 non-null   int64  
 1   Order_ID     9994 non-null   object 
 2   Order_Date   9994 non-null   object 
 3   Ship_Date    9994 non-null   object 
 4   Ship_Mode    9994 non-null   object 
 5   Customer_ID  9994 non-null   object 
 6   Product_ID   9994 non-null   object 
 7   Sales        9994 non-null   float64
 8   Quantity     9994 non-null   int64  
 9   Discount     9994 non-null   float64
 10  Profit       9993 non-null   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 859.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer_ID    793 non-null    object
 1   Customer_Name  793 non-null    object
 2   Se

In [6]:
df_RFMOverall

,CustomerID,CustomerName,Recency Value,Frequency Value,Monetery Value,R-Score,F-Score,M-Score,RFM_Overall,Segment
0,CG-12520,Claire Gute,3219,3,1148.780001,1,1,2,112,Lost customers
1,DV-13045,Darrin Van Huff,2900,5,1119.482999,5,2,2,522,Promising
2,SO-20335,Sean O'Donnell,2910,6,2602.575532,4,3,3,433,Potential Loyalist
3,BH-11710,Brosina Hoffman,2904,8,6255.350936,5,4,5,545,Champions
4,AA-10480,Andrew Allen,3140,4,1790.512015,1,1,3,113,Cannot Lose Them
...,...,...,...,...,...,...,...,...,...,...
798,CJ-11875,Carl Jackson,3246,1,16.520000,1,1,1,111,Lost customers
799,RS-19870,Roy Skaria,2900,2,22.328000,5,1,1,511,New Customers
800,SC-20845,Sung Chung,3012,2,280.631994,2,1,1,211,Hibernating customers
801,RE-19405,Ricardo Emerson,3978,1,48.360001,1,1,1,111,Lost customers


In [7]:
import pyodbc

# --- Connect ---
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)"

table_name = "data_RFM_Insights"

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
cols = ", ".join([f"[{c}] {dtype_map(df_RFMOverall[c].dtype)}" for c in df_RFMOverall.columns])
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(df_RFMOverall.columns))})", df_RFMOverall.values.tolist())
conn.commit()